# Velkommen til workshop i nevrale nettverk hos Husbanken

Det første vi må gjøre er å importere bibliotekene vi trenger.

In [ ]:
from keras.datasets import mnist #Dette er datasettet - vi slipper å laste opp, convertere erc.
import keras
import matplotlib.pyplot as plt
import numpy as np
import time

#Importer lag og modell fra Keras
from keras.layers import Dense 
from keras.models import Sequential 

In [ ]:
# Hjelpefunksjoner for å nullstille grafen, visualisere litt, prediksjon og utskrift av test-data.
from keras import backend as K

def init_graph():
    K.clear_session()
    
def plot_graph(history, loss, accuracy):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='best')
    plt.show()

    print(f'Test loss: {loss:.3}')
    print(f'Test accuracy: {accuracy:.3}')
    
def predictDigit(n):
    print("Algoritmen gjettet ", np.argmax(model.predict(X_test[n:n+1])))
    
def printDigit(i):
    fig = plt.figure()
    plt.imshow(x_test[i], cmap='gray', interpolation='none')
    print("Riktig svar er: ", y_test[i])
    
def call_eval(p):
    predictDigit(p)
    printDigit(p)

### Det er vanlig å dele opp datasettet i to: ett til å trene algoritmen og ett annet for å teste den

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() #Denne returnerer to tupler med to lister i hver

In [ ]:
# Prøv å få et forhold til hvordan dataene ser ut. Her er to vektorer hvor hvert element er en matrise
print("Training data shape: ", x_train.shape)
print("Test data shape", x_test.shape)

In [ ]:
# Hvordan set dataene ut? Legg merke til merkelappene som tilhører hvert bilde.
fig = plt.figure()
n = 9
for i in range(n):
  plt.subplot(3,3,i+1)
  plt.tight_layout()
  plt.imshow(x_train[i], cmap='gray', interpolation='none')
  plt.title("{}".format(y_train[i]))
  plt.xticks([])
  plt.yticks([])

In [ ]:
image_vector_size = 28*28
X_train = x_train.reshape(x_train.shape[0], image_vector_size)
X_test = x_test.reshape(x_test.shape[0], image_vector_size)

Når det gjelder output, så vil nettverket gi oss sannsynligheten for at bildet er et spesifikt tall. Vi har ti ou-nevroner, hvor den første representerer 0, den andre 1 osv. For nettverket gir selve grunntallene ingen mening. Vi kan forvente at output blir kodet som _one-hot_

In [ ]:
num_classes = 10 #Hvor mange klasser har vi (i.e. tall som skal klassifiseres)

Y_train = keras.utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
print("Training lables 0 - %d as one-hot encoded vectors:\n" % n, Y_train[:n])

# Nå skal vi endelig lage modellen (nettverket)

In [ ]:
input_size = image_vector_size #Hvor mange piksler skal inn av gangen? Hele bildet?

model = Sequential() #Vi lager en sekvensiell modell
# Vi legger til ett "dense-lag". Input_shape forteller hvordan form input tar.
# Activation: hvilken funksjon skal vi ha? Hvilken parameter angir antall noder i "hidden layer"?
model.add(Dense(units=16, activation='sigmoid', input_shape=(input_size,)))
# Med Keras er det ikke nødvendig å angi input-shape for lagene n > 1
model.add(Dense(units=num_classes, activation='sigmoid'))

In [ ]:
# Hvor mange parametre blir dette totalt? Blyant og papir!
model.summary()
# Hvor mange fikk du? Hvorfor bommet du?

# Modellen må kompileres (i.e. settes ihop med loss-funksjon, optimiseringsalgoritme og annet)

In [ ]:
# Vi bruker stochastic gradient descent, cross entrophy for å "måle" informasjonslikheten mellom y og yhat
model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])
# Vi kjører nå fit-funksjonen. Denne tilpasser nettverket ditt til distribusjonen i trenings-settet.
# Batch-size: Hvor mange samples skal vi kjøre før vi oppgraderer gradienten? Når har vi behov for å justere 
# dette hyper-parameteret? Hint: (e.g.) NVidia GTX 1080 Ti Turbo med 11 G ram!
# Den er raskere enn "vanlig" og presisjonen er ikke så langt
# borte fra gd. Når er det en fordel med SGD? Finn.no og Amazon? <- Når de skal anbefale deg noe, har de da tid og
# ressurser til å justere modellen med all data de har? (Kanskje Amazon, men ihvertfall ikke Finn).
# Kjør gjerne med verbose = True.
# Validation: validation_split bestemmer hvor mye av dataen som skal brukes til validering. (Se nedenfor)
# Hva returnerer .fit? Den returnerer et history-objekt. Hva inneholder dette? prøv dir(history)
history = model.fit(X_train, Y_train, batch_size=128, epochs=5, verbose=False, validation_split=.1)
# Loss og accuracy: hva er egentlig dette? Metrikker for å evaluere modellen mot et testsett?
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)


In [ ]:
plot_graph(history, loss, accuracy)

In [ ]:
# ADVARSEL! Bruk denne kun dersom du skal nullstille grafen. Alle parametre blir slettet! 
#init_graph()

# På tide å se med egne øyne

In [ ]:
call_eval(54) #Sett inn tall som fungerer for indeks inn i test-settet.

# På tide å justere nettverkets hyperparametre!

#### Du skjønner sikkert at 82 prosent nøyaktighet ikke er bra nok. Hvis du har 1 000 000 siffer, skal du da ha 180 000 feil-klassifiseringer? Se hva du kan gjøre for å øke nøyaktigheten.

Modellen:
* antall lag
* antall noder (eller aktiveringer, som det også kalles)
* hvilken aktiveringsfunksjon er best? Sjekk ut:
    * ReLu LeakyReLu
    * Tangens Hyperbolicus
    * SoftMax
* Spesielt interesserte kan også sjekke ut
    * PReLu
    * ArcTan
    * SoftPlus
    
Hvorfor er det ingen god ide å bruke en lineær aktiveringsfunksjon (aka. identitetsfunksjonen)?

#### Så til trening av modellen
Hva slags optimiseringsalgoritme skal du bruke?
* SGD
* RMSProp
* RMSMomentum
* AdaProp
* AdaDelta
* AdaGrad
* Adam
* Nadam
* Nesterov

Hvor mange epochs? 

Hvordan vil du initialisere vektene? Dette har stor betydning! Du kan også lese om å bryte symmetrien, som er fullstendig avgjørende for et bra resultat. Prøv dir(keras.initializers) eller keras.initializers. <- (trykk tab)


$model.add(Dense(64,
                kernel\_initializer='random\_uniform',
                bias\_initializer='zeros'))$
                
Les om modellen i Keras-dokumentasjonen, som forøvrig er fremragende!

https://keras.io/models/model/

Gå tilbake, reset grafen, endre hyper-parametrene og se om du klarer å forbedre resultatet.

### Nevrale nettverk har en del utfordringer (som de fleste andre algoritmer). 

Hva tror du skjer med nøykaktigheten dersom en sample fra trenings-settet klarer å snike seg inn i test-settet? Hvis vi hadde testet med 10 prosent av trenings-settet?

Hva om vi trener for mye på ett trenings-sett? Har noen sett en lineær regresjon med polynomiske variable (som egentlig bare er et spesialtilfelle av lineær regressjon) som har meget store parametre? Hva skjer med training error? Hvordan generaliserer den? Vi får "høy bias", dvs. kurven er godt tilpasset dataene være. Dette er ikke ønskelig. På den andre siden, dersom vi ikke trener godt nok eller har for lite trenings-sett, kan vi få altfor høy varianse. Som alt annet i livet, er dette et spørsmål om optimalisering. 

Bias/variance-tradeoff!